#### Урок 2. Парсинг HTML. BeautifulSoup, MongoDB

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайтов Superjob и HH. Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
* Наименование вакансии.
* Предлагаемую зарплату (отдельно минимальную, максимальную и валюту).
* Ссылку на саму вакансию.
* Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [60]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [61]:
#https://hh.ru/search/vacancy?area=1&fromSearchLine=true&st=searchVacancy&text=python

In [62]:
def hh_search(search_vacancy):

    main_link = 'https://hh.ru'
    params = {'text': search_vacancy, 'page': ''}
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36'}
    response = requests.get(main_link + '/search/vacancy?area=1&fromSearchLine=true&st=searchVacancy&text=', params = params, headers = headers)

    vacancy_data = []

    if response.status_code == 200:
        dom = bs(response.text, 'lxml')

        page_block = dom.find('div', {'data-qa': 'pager-block'})
        if not page_block:
            last_page = '1'
        else:
            last_page = int(page_block.find_all('a', {'class':'HH-Pager-Control'})[-2].getText())

    for page in range(0,last_page):
        params['page'] = page
        response = requests.get(main_link + '/search/vacancy?area=1&fromSearchLine=true&st=searchVacancy&text=', params = params, headers = headers)

        if response.status_code == 200:
            dom = bs(response.text, 'lxml')

            vacancy_items = dom.find_all('div', {'data-qa': 'vacancy-serp__vacancy'})

            for item in vacancy_items:
                vacancy_data.append(get_vacancy_data(item))
    return vacancy_data


def get_vacancy_data(item):

    vacancy_data = {}

    name_vacancy = item.find('span',{'class':'g-user-content'}).getText()
    vacancy_data['name_vacancy'] = name_vacancy

    get_link = item.find('span', {'class':'g-user-content'})
    vacancy_link = get_link.findChild()['href']
    vacancy_data['vacancy_link'] = vacancy_link

    company_name = item.find('div',{'class':'vacancy-serp-item__meta-info'}).getText()
    vacancy_data['company_name'] = company_name

    vacancy_city = item.find('span', {'class': 'vacancy-serp-item__meta-info'}).getText().split(', ')[0].replace(u'\xa0', u'')
    vacancy_data['vacancy_city'] = vacancy_city

    salary = item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary = salary.getText().replace('\xa0', '')
        salary = re.split(' |-', salary)

        try:
            salary_currency = salary[2]
        except IndexError:
            salary_currency = salary[1]
        if salary[0] == 'до':
            salary_min = None
            salary_max = int(salary[1])
        elif salary[0] == 'от':
            salary_min = int(salary[1])
            salary_max = None
        else:
            salary_min = int(salary[0])
            try:
                salary_max = int(salary[1])
            except ValueError:
                salary_max = None

    vacancy_data['salary_min'] = salary_min
    vacancy_data['salary_max'] = salary_max
    vacancy_data['salary_currency'] = salary_currency

    return vacancy_data


def parser_vacancy(vacancy_search):
    vacancy_data = []
    vacancy_data.extend(hh_search(vacancy_search))

    df = pd.DataFrame(vacancy_data)
    return df



In [66]:
vacancy_search = 'менеджер по закупкам'
df = parser_vacancy(vacancy_search)

In [67]:
df

,name_vacancy,vacancy_link,company_name,vacancy_city,salary_min,salary_max,salary_currency
0,Менеджер по закупкам и логистике,https://hh.ru/vacancy/40220888,ООО Мебель-Тон,Москва,105118.0,105118.0,руб.
1,Менеджер по закупкам (удаленная работа),https://hh.ru/vacancy/40169481,Интернет магазин Mebelion.ru (Мебелион.Ру),Москва,23000.0,NaN,руб.
2,Менеджер по закупкам в интернет-магазин,https://hh.ru/vacancy/40136207,Битуби Системс,Москва,50000.0,70000.0,руб.
3,Менеджер по закупкам,https://hh.ru/vacancy/40263658,Фуд-холл (ООО УК Первый Ранг),Москва,150000.0,NaN,руб.
4,Менеджер по закупкам,https://hh.ru/vacancy/40353955,ООО ВТОРОБОРУДОВАНИЕ,Москва,70000.0,200000.0,руб.
...,...,...,...,...,...,...,...
948,Специалист по закупкам (строительство),https://hh.ru/vacancy/40306314,ООО АФС,Москва,50000.0,NaN,руб.
949,Менеджер по закупкам,https://hh.ru/vacancy/40405843,ООО БМП-СИМИРА,Москва,45000.0,80000.0,руб.
950,Менеджер по закупкам,https://hh.ru/vacancy/40255432,ООО ЭнергоРемонтСервис,Москва,55000.0,NaN,руб.
951,"Менеджер по закупкам электротехнических, метал...",https://hh.ru/vacancy/39925157,ООО Светорезерв,Москва,30000.0,35000.0,руб.
